$\forall k \in \mathcal{K} : \Phi^k (\theta_{pt}) = \min_{w_{pt}^k, v_{jt}^k, z_{jt}^k} - \left( \sum_{j,t} \gamma_{jt} z_{jt}^k - \sum_{p,t} \sigma_{pt} w_{pt}^k - \sum_{j,t} \varphi_{jt}^k v_{jt}^k \right)$\
$w_{pt}^k \leq \theta_{pt}, \quad \forall p \in \mathcal{P}, \, t \in \mathcal{T}$\
$v_{jt}^k \leq A_{jt}^k, \quad \forall j \in \mathcal{J}, \, t \in \mathcal{T}$\
$z_{jt}^k \leq D_{jt}^k, \quad \forall j \in \mathcal{J}, \, t \in \mathcal{T}$\
$-w_{pt}^k \leq 0, \quad \forall p \in \mathcal{P}, \, t \in \mathcal{T}$\
$-v_{jt}^k \leq 0, \quad \forall j \in \mathcal{J}, \, t \in \mathcal{T}$\
$-z_{jt}^k \leq 0, \quad \forall j \in \mathcal{J}, \, t \in \mathcal{T}$\
$v_{jt}^k - z_{jt}^k - \sum_p \left( \eta_{jp} - \mu_{jp} \right) w_{pt}^k = 0, \quad \forall j \in \mathcal{J}, \, t \in \mathcal{T}$\
$\theta_{pt} \in \left[ 0, Q_{pt}^U \right], \quad \forall p \in \mathcal{P}, \, t \in \mathcal{T}$

In [67]:
#%%
# Importing the necessary lib
import pandas as pd
import seaborn as sns
import numpy as np
#import pyreadr as py # library to read .Rdata files in python

import matplotlib.pyplot as plt

In [68]:
n_z=1
n_w=3
n_v =2
n_d=3
# number of variables (z, omega, nu, d)

In [69]:
eta_jp =  np.array([[1.11,0,0],[0,1.22,1.05],[0,0,0]])
mu_jp = np.array([[0,0,0],[1,0,0],[0,1,1]])

In [70]:
n_t = 12 # number of params (q, gamma, sigma. sai, A, D)

n_q = 3
n_gamma = 1
n_sigma = 3
n_phi = 2
n_A = 2
n_D = 1

In [71]:
# objecitve funciotn
c = np.zeros((n_z + n_w + n_v + n_d)).reshape(-1,1)

H = np.zeros((n_t , n_z + n_w + n_v + n_d))

H[n_q:n_q+n_gamma,0:n_z] = np.eye(n_gamma)

H[n_q+n_gamma:n_q+n_gamma+n_sigma,n_z:n_z+n_w] = -np.eye(n_sigma)

H[n_q+n_gamma+n_sigma:n_q+n_gamma+n_sigma+n_phi,n_z+n_w : n_z+n_w+n_v] = -np.eye(n_phi)

H = -H

In [72]:
# dual constrains equaltiy constraitns d = q

#d == q
A1 = np.zeros((n_d,n_z + n_w + n_v + n_d))
A1[:,n_z + n_w + n_v:] = np.eye(n_d)

b1 = np.zeros(n_d).reshape(-1,1)

F1 = np.zeros((n_d,n_t))
F1[:,:n_q] = np.eye(n_q)

# #d>=q
# A2 = np.zeros((n_j,n_x*n_j))
# A2[:,3*n_j:] = -np.eye(n_j)

# b2 = np.zeros(n_j).reshape(-1,1)

# F2 = np.zeros((n_j,n_t*n_j))
# F2[:,:n_j] = -np.eye(n_j)


In [73]:
# omega <= d
A3 = np.zeros((n_w,n_z + n_w + n_v + n_d))
A3[:,n_z:n_z+n_w] = np.eye(n_w)
A3[:,n_z + n_w + n_v:] = -np.eye(n_d)

b3 = np.zeros(n_w).reshape(-1,1)

F3= np.zeros((n_w,n_t))

In [74]:
# nu <= A

A4 = np.zeros((n_v,n_z + n_w + n_v + n_d))
A4[:,n_z + n_w:n_z + n_w + n_v] = np.eye(n_v)

b4 = np.zeros(n_v).reshape(-1,1)

F4 = np.zeros((n_v,n_t))
F4[:,n_q+n_gamma+n_sigma+n_phi:n_q+n_gamma+n_sigma+n_phi+n_A] = np.eye(n_A)

In [75]:
# z <= D

A5 = np.zeros((n_z,n_z + n_w + n_v + n_d))
A5[:,:n_z] = np.eye(n_z)

b5 = np.zeros(n_z).reshape(-1,1)

F5 = np.zeros((n_z,n_t))
F5[:,n_q+n_gamma+n_sigma+n_phi+n_A:] = np.eye(n_D)

In [76]:
-(eta_jp-mu_jp)[:-1,:]

array([[-1.11, -0.  , -0.  ],
       [ 1.  , -1.22, -1.05]])

In [77]:
# equality constraints

A7 = np.zeros((n_v,n_z + n_w + n_v + n_d))

A7[:,n_z:n_z + n_w] = -(eta_jp-mu_jp)[:-1,:]

A7[:,n_z + n_w:n_z + n_w + n_v] = np.eye(n_v)

b7 = np.zeros(n_v).reshape(-1,1)

F7 = np.zeros((n_v,n_t))


In [78]:
A8 = np.zeros((n_z,n_z + n_w + n_v + n_d))

A8[:,n_z:n_z + n_w] = -(eta_jp-mu_jp)[-1,:]

A8[:,:n_z] = -np.eye(n_z)

b8 = np.zeros(n_z).reshape(-1,1)

F8 = np.zeros((n_z,n_t))


In [79]:
# all x >= 0
A6 = -np.eye(n_z + n_w + n_v + n_d)

b6 = np.zeros(n_z + n_w + n_v + n_d).reshape(-1,1)

F6 = np.zeros((n_z + n_w + n_v + n_d,n_t))

In [80]:


# A8 = np.zeros((n_j,n_x*n_j))
# A8[:,:1*n_j] = np.eye(n_j)
# A8[:,1*n_j:2*n_j] = (eta_jp-mu_jp)
# A8[:,2*n_j:3*n_j] = -np.eye(n_j)

# b8 = np.zeros(n_j).reshape(-1,1)

# F8 = np.zeros((n_j,n_t*n_j))


In [81]:
A = np.vstack((A1,A7,A8,A3,A4,A5,A6))
b = np.vstack((b1,b7,b8,b3,b4,b5,b6))
F = np.vstack((F1,F7,F8,F3,F4,F5,F6))

In [82]:
CRa1 = np.eye(n_t)
CRb1 = np.array([100,100,100,100,2,2,2,15,35,20,70,100]).reshape(-1,1) #(q, gamma, sigma. sai, A, D)

CRa2 = -np.eye(n_t)
CRb2 = np.zeros(n_t).reshape(-1,1)

CRa = np.vstack((CRa1,CRa2))
CRb = np.vstack((CRb1,CRb2))

In [83]:
A.shape

(21, 9)

In [84]:
# import scipy.io as sio
# data = {
#     'H': H,
#     'c': c,
#     'A': A,
#     'b': b,
#     'F': F,
#     'CRA': CRa,
#     'CRb': CRb
# }

# # Save the dictionary to a .mat file
# # sio.savemat('chem_process_primal.mat', data)
# # np.savez('data_NN7_Oct_19.npz', **data)

In [85]:
from ppopt.mpqp_program import MPLP_Program

A = A
b = b
c = c.reshape(-1,1)
F = F
CRa = CRa
CRb = CRb.reshape(-1,1)
H = H.T

prog = MPLP_Program(A, b, c, H, CRa, CRb, F,equality_indices=[0,1,2,3,4,5])

In [86]:
F.shape

(21, 12)

In [87]:
prog.process_constraints()

In [88]:
prog.display_latex()

$$\min_{x}\left[\begin{matrix}0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]^T\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right] + \left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\\\theta_{9}\\\theta_{10}\\\theta_{11}\end{matrix}\right]^{T}\left[\begin{matrix}0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]^{T}\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right]$$
$$\left[\begin{matrix}0 & 0.7071 & 0 & 0 & 0 & 0 & -0.

$$\min_{x}\left[\begin{matrix}0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]^T\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right] + \left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\\\theta_{9}\\\theta_{10}\\\theta_{11}\end{matrix}\right]^{T}\left[\begin{matrix}0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]^{T}\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right]$$
$$\left[\begin{matrix}0 & 0.7071 & 0 & 0 & 0 & 0 & -0.7071 & 0 & 0\\0 & 0 & 0.7071 & 0 & 0 & 0 & 0 & -0.7071 & 0\\0 & 0 & 0 & 0.7071 & 0 & 0 & 0 & 0 & -0.7071\\0 & 0 & 0 & 0 & 0.7071 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0.7071 & 0 & 0 & 0\\0.7071 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right]\leq\left[\begin{matrix}0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]+\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.7071 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.7071 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.7071\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\\\theta_{9}\\\theta_{10}\\\theta_{11}\end{matrix}\right]$$
$$\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0.7071 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.7071 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.7071\\0 & -0.743 & 0 & 0 & 0.6693 & 0 & 0 & 0 & 0\\0 & 0.4667 & -0.5694 & -0.49 & 0 & 0.4667 & 0 & 0 & 0\\-0.5774 & 0 & 0.5774 & 0.5774 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\\x_{8}\end{matrix}\right]=\left[\begin{matrix}0\\0\\0\\0\\0\\0\end{matrix}\right]+\left[\begin{matrix}0.7071 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0.7071 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0.7071 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\\\theta_{9}\\\theta_{10}\\\theta_{11}\end{matrix}\right]$$
$$\left[\begin{matrix}1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0\\-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\\\theta_{9}\\\theta_{10}\\\theta_{11}\end{matrix}\right]\leq\left[\begin{matrix}100.0\\100.0\\100.0\\100.0\\2.0\\2.0\\2.0\\15.0\\35.0\\20.0\\70.0\\100.0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]$$

In [89]:
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
solution = solve_mpqp(prog, mpqp_algorithm.combinatorial)


In [90]:
len(solution)

87

In [91]:
import pickle

#Save the solution dictionary to a file
with open('solution.pkl', 'wb') as f:
    pickle.dump(solution, f)

In [92]:
solution.critical_regions[1].A.shape

(9, 12)

In [93]:
solution.critical_regions

[Critical region with active set [0, 1, 2, 3, 4, 5, 6, 7, 8]
 The Omega Constraint indices are [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
 The Lagrange multipliers Constraint indices are [6, 7, 8]
 The Regular Constraint indices are [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
   x(θ) = Aθ + b 
  λ(θ) = Cθ + d 
   Eθ <= f
  A = [[-2.02133188e-16  1.00000000e+00  1.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  1.87312984e-16 -3.60876679e-16  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-1.99606496e-16  1.00000000e+00 -2.23515391e-16  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0

In [94]:
solution.evaluate(np.array([12,13.15,60,31.7,0.48,0.72,0.6,5.81,13.9,8.71,29,43.6]).reshape(-1,1))

array([[3.50922351e+01],
       [7.84684685e+00],
       [6.55710401e-14],
       [3.50922351e+01],
       [8.71000000e+00],
       [2.90000000e+01],
       [1.20000000e+01],
       [1.31500000e+01],
       [6.00000000e+01]])

In [95]:
solution.get_region(np.array([12,13.15,60,3.8,0.48,0.72,0.6,5.81,13.9,8.71,29,43.6]).reshape(-1,1)).C@np.array([12,13.15,60,3.8,0.48,0.72,0.6,5.81,13.9,8.71,29,43.6]).reshape(-1,1) + solution.get_region(np.array([12,13.15,60,3.8,0.48,0.72,0.6,5.81,13.9,8.71,29,43.6]).reshape(-1,1)).d

array([[ 2.49603812e-15],
       [ 2.47580345e-15],
       [ 7.06178024e-16],
       [-8.68026444e+00],
       [-1.48465568e+01],
       [-1.36408614e+01],
       [ 4.07555500e+00],
       [ 1.29794700e+00],
       [ 6.97090000e+00]])

In [96]:
solution.evaluate_objective(np.array([50,50.15,60,3.8,0.48,0.72,0.6,5.81,13.9,8.71,29,43.6]).reshape(-1,1))

-2.060043392991632e-13

In [97]:
(1/np.linalg.norm(np.block([A, -F]),axis=1, keepdims=True))

array([[0.70710678],
       [0.70710678],
       [0.70710678],
       [0.66933445],
       [0.46671427],
       [0.57735027],
       [0.70710678],
       [0.70710678],
       [0.70710678],
       [0.70710678],
       [0.70710678],
       [0.70710678],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ]])

In [98]:
A_list = []
b_list = []

C_list = []
d_list=[]

E_list = []
f_list = []

active_list = []

for i in solution.critical_regions:
    A_list.append(i.A)
    b_list.append(i.b)
    
    C_list.append(i.C)
    d_list.append(i.d)
    
    E_list.append(i.E)
    f_list.append(i.f)
    
    active_list.append(i.active_set)

In [99]:
from scipy.io import savemat
data = {
    'A_list' : A_list,
    'b_list' : b_list,
    'C_list' : C_list,
    'd_list' : d_list,
    'E_list' : E_list,
    'f_list' : f_list,
    'active_list':active_list,
    'c': c.reshape(-1,1),
    'H': H.T,
    'multiplier': (1/np.linalg.norm(np.block([A, -F]),axis=1, keepdims=True))[:3]
}

savemat("sp_primal_chemical_process_simp.mat", data)